In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [2]:
import os
from pinecone import Pinecone as pcn
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from IPython.display import Markdown
import gradio as gr
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_openai import ChatOpenAI 

/home/acrisvall/recipes_rag/.venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### 01 Get API Keys

In [3]:
api_key = os.environ.get("PINECONE_API_KEY")
openai_api_key = os.getenv('OPENAI_API_KEY')

In [4]:
# configure client
pc = pcn(api_key=api_key)
index_name = 'recipes-index'
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1106}},
 'total_vector_count': 1106}

### 02 Set Embedding Model

In [5]:
embed_model = "text-embedding-3-small"
embed = OpenAIEmbeddings(model=embed_model)

In [6]:
text_field = "description"

vectorStore = PineconeVectorStore(
  index, embed, text_field
)

In [22]:
def get_recipe(query):# completion llm  
    llm = ChatOpenAI(  
        model_name='gpt-4o-mini',  
        temperature=0.0  
    ) 

    retriever = vectorStore.as_retriever(search_type="similarity", k=3)
    # 2. Incorporate the retriever into a question-answering chain.
    system_prompt = (
    """
        Always add a kind and friendly message according to the context at the beginning of the response.
        Take the information from context[0]["metadata"]
        Then follow this format
        Title: [Title of the Recipe]
        Description: [Short Description of the Recipe]
        Preparation Time: [Preparation Time]
        Cooking Time: [Cooking Time]
        Difficulty: [Difficulty Level]
        Serves: [Number of Servings]
        Diet Type: [Diet Type]
        Nutrition: calories context[0]["metadata"]["calories"] | fat context[0]["metadata"]["fat"] | protein context[0]["metadata"]["protein"] | fiber context[0]["metadata"]["fibre"]
        Ingredients: [List of Ingredients]
        Instructions: [Step-by-step Cooking Instructions]

        If you don't have any general information, just respond with "I don't know!"

        {context}
        {{metadata}}
        """
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )

    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    response = rag_chain.invoke({"input": query})
    return response["answer"]

## Gradio App

In [23]:
demo = gr.Interface(
    fn=get_recipe,
    inputs=["text"],
    outputs=["text"],
)

demo.launch()

Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


{'input': 'what recipe can i cook with chicken', 'context': [Document(id='107', metadata={'calories': '506', 'carbs': '40g', 'cook_time': '20 mins', 'diet_type': 'Dairy-free', 'difficulty': 'Easy', 'fat': '19g', 'fibre': '4g', 'id': '107', 'ingredients': '320g packchicken mini breast fillets, 2½ tbsplight soy sauce, vegetable oil,for frying, 1red pepper,deseeded and thinly sliced, 1green pepper,deseeded and thinly sliced, 3 tbspcornflour, 3 tbspsweet chilli sauce, 1 tbspgarlic and ginger paste, 1 tbspsesame oil, 2spring onions,trimmed and finely sliced', 'instructions': 'Slice the chicken into thin strips. Pour over 2 tbsp of the soy and marinate in the fridge, covered, for 1 hr. Meanwhile, heat 1 tbsp of the oil in awokor deep frying pan over a medium heat and cook the peppers for 3-4 mins until just beginning to soften. Remove with a slotted spoon and set aside. Spread the cornflour out in a shallow bowl. Season, then add the marinated chicken strips and toss to coat well. Fill a wok